### Dashboard test

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
import pyspark
import pandas as pd
import warnings
import gradio as gr
warnings.filterwarnings('ignore', category=DeprecationWarning)
from pyspark.sql.types import StringType,TimestampType
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
ss_name = 'Lectura de datos Dashboard'
wg_conn = "spark.kerberos.access.hadoopFileSystems"
db_conn = "abfs://data@datalakesii.dfs.core.windows.net/"

spark = SparkSession.builder \
      .appName(f"Ejecucion algoritmo {ss_name}")  \
      .config(wg_conn, db_conn) \
      .config("spark.executor.memory", "6g") \
      .config("spark.driver.memory", "12g")\
      .config("spark.executor.cores", "4") \
      .config("spark.executor.instances", "5") \
      .config("spark.driver.maxResultSize", "12g") \
      .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

spark.conf.set("spark.sql.parquet.enableVectorizedReader","false")
spark.conf.set("spark.sql.parquet.int96RebaseModeInRead", "CORRECTED")
spark.conf.set("spark.sql.parquet.int96RebaseModeInWrite", "CORRECTED")
spark.conf.set("spark.sql.parquet.datetimeRebaseModeInRead", "CORRECTED")
spark.conf.set("spark.sql.parquet.datetimeRebaseModeInWrite", "CORRECTED")

Setting spark.hadoop.yarn.resourcemanager.principal to yasser.nanjari


In [3]:
contribuyentes=spark.read.parquet("abfs://data@datalakesii.dfs.core.windows.net/DatosOrigen/lr-629/riesgo_fraude/proyecto_visualizacion_racionales_positivos/data_contribuyentes").toPandas()
iva=spark.read.parquet("abfs://data@datalakesii.dfs.core.windows.net/DatosOrigen/lr-629/riesgo_fraude/proyecto_visualizacion_racionales_positivos/data_iva_dashboard").toPandas()
contribuyente_iva=spark.read.parquet("abfs://data@datalakesii.dfs.core.windows.net/DatosOrigen/lr-629/riesgo_fraude/proyecto_visualizacion_racionales_positivos/data_iva_contribuyentes").toPandas()
spark.stop()

In [4]:
contribuyentes[contribuyentes['SOCIEDAD']==1]

,RUT_UNICO,SOCIEDAD,CONTADOR,NATURAL,REPRESENTANTE,Contaminacion,Total_pago_f29,IVA_neto,Unidad_regional,Numero_documentos,...,Familiares,Personas_naturales_socios,Tasa_familiaridad,Grupo_economico_inicial,Grupo_louvain,Trabajadores_honorarios,Trabajadores_dependientes,Patrimonio_2022,ultimo_periodo_remanente_contaminado,remanente_cont
1,++0YvM0f0Bcsgr/4gsiZFA==,1,0,0,0,NaN,NaN,NaN,None,NaN,...,NaN,NaN,NaN,NaN,1012197,NaN,NaN,NaN,None,NaN
14,++6WPuZR8eH4QeBHRcGTcQ==,1,0,0,0,NaN,NaN,NaN,None,NaN,...,NaN,NaN,NaN,NaN,1148044,NaN,NaN,NaN,None,NaN
17,++6sRlljUAs2fd6lmPoT9g==,1,0,0,0,NaN,NaN,NaN,None,NaN,...,NaN,NaN,NaN,NaN,767934,NaN,NaN,NaN,None,NaN
23,++8yrddEhyjEVxjMJ1J0bQ==,1,0,0,0,0.048,0.0,48989.0,8VA UNIDAD REGIONAL BIO BIO,NaN,...,0.0,1.0,0.0,NaN,64251,NaN,NaN,1.785715e+07,None,NaN
35,++BSKnfO41BOVY1qyzbo+Q==,1,0,0,0,NaN,NaN,NaN,None,NaN,...,NaN,NaN,NaN,NaN,25722,NaN,NaN,NaN,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24206308,zzACenkIoQteyXL+9cKy9A==,1,0,0,0,0.050,0.0,8930387.0,5TA UNIDAD REGIONAL VALPARAISO,NaN,...,0.0,1.0,0.0,NaN,1200459,12.0,5.0,2.354601e+07,None,NaN
24206311,zzElu5N90axkB7yh2kZoeA==,1,0,0,0,0.049,0.0,1134379.0,8VA UNIDAD REGIONAL BIO BIO,38.0,...,2.0,2.0,100.0,NaN,28109,NaN,NaN,NaN,None,NaN
24206313,zzGqQjqRmObVgYPuzbkIug==,1,0,0,0,0.050,195668.0,1473471.0,SANTIAGO NORTE,NaN,...,0.0,2.0,0.0,NaN,1574706,0.0,1.0,6.032639e+07,None,NaN
24206325,zzUGMc2Gse5352xabEUlJA==,1,0,0,0,NaN,NaN,NaN,None,NaN,...,2.0,2.0,100.0,NaN,49567,NaN,NaN,1.598900e+09,None,NaN


In [5]:
import pandas as pd
import gradio as gr
import plotly.express as px

def format_money_columns(df, columns):
    """
    Formatear las columnas especificadas con el signo $ y separar miles.
    """
    for col in columns:
        df[col] = df[col].apply(lambda x: f"${x:,.0f}" if pd.notnull(x) else "")
    return df

def create_iva_comparison_chart(iva_data, columns, title):
    """
    Crear diagrama circular comparativo entre IVA intragrupo y extragrupo mostrando porcentajes.
    """
    iva_data['Valor'] = iva_data['Valor'].replace({'\$': '', ',': ''}, regex=True).astype(float)
    iva_data['Porcentaje'] = iva_data['Valor'] / iva_data['Valor'].sum() * 100
    colors = ['#FFA500', '#1E90FF']  # Naranja y Azul

    fig = px.pie(iva_data, names='Atributo', values='Porcentaje',
                 title=title, 
                 color_discrete_sequence=colors)

    fig.update_traces(textposition='inside', textinfo='percent+label', 
                      texttemplate='%{percent:.2f}% (%{label})')
    fig.update_layout(showlegend=True)

    return fig

def create_iva_receptor_comparison_chart(iva_data, columns, title):
    """
    Crear diagrama circular comparativo entre IVA intragrupo y extragrupo mostrando porcentajes para receptores.
    """
    iva_data['Valor'] = iva_data['Valor'].replace({'\$': '', ',': ''}, regex=True).astype(float)
    iva_data['Porcentaje'] = iva_data['Valor'] / iva_data['Valor'].sum() * 100
    colors = ['#32CD32', '#FF4500']  # Verde y Rojo

    fig = px.pie(iva_data, names='Atributo', values='Porcentaje',
                 title=title, 
                 color_discrete_sequence=colors)

    fig.update_traces(textposition='inside', textinfo='percent+label', 
                      texttemplate='%{percent:.2f}% (%{label})')
    fig.update_layout(showlegend=True)

    return fig

def filter_data(cont_rut):
    
    if cont_rut:
        cont_rut = cont_rut.replace(" ", "")
        filtered_df = contribuyentes[contribuyentes["RUT_UNICO"] == cont_rut]
        
        if filtered_df.empty:
            return pd.DataFrame(columns=contribuyentes.columns), pd.DataFrame(columns=iva.columns), pd.DataFrame(columns=contribuyente_iva.columns), None, None, None, None
        
        columnas_ordenadas = ['RUT_UNICO', 'Unidad_regional'] + [col for col in filtered_df.columns if col not in ['RUT_UNICO', 'Unidad_regional']]
        filtered_df = filtered_df[columnas_ordenadas]
        
        money_columns_contribuyentes = ['Total_pago_f29', 'IVA_neto', 'Patrimonio_2022', 'remanente_cont']
        filtered_df = format_money_columns(filtered_df, money_columns_contribuyentes)

        filtered_df_vertical = filtered_df.melt(var_name="Atributo", value_name="Valor")
        grupo_louvain = filtered_df['Grupo_louvain'].iloc[0]
        
        iva_filtered_df = iva[iva["com"] == grupo_louvain]
        money_columns_iva = ['emision_extragrupo', 'recepcion_extragrupo','recepcion_intragrupo', 'emision_intragrupo']
        iva_filtered_df = format_money_columns(iva_filtered_df, money_columns_iva)
        
        iva_filtered_vertical = iva_filtered_df.melt(var_name="Atributo", value_name="Valor")
        
        # Gráficos de emisión
        chart_iva_louvain = create_iva_comparison_chart(
            iva_filtered_vertical[iva_filtered_vertical['Atributo'].isin(['emision_intragrupo', 'emision_extragrupo'])],
            ['emision_intragrupo', 'emision_extragrupo'],
            "Comparación de emisión IVA Intragrupo vs Extragrupo - Grupo Louvain"
        )
        
        contribuyente_iva_filtered = contribuyente_iva[contribuyente_iva["contribuyente"] == cont_rut]
        money_columns_contribuyente_iva = [
            'total_iva_intragrupo_emisor', 'total_iva_extragrupo_emisor', 
            'total_iva_intragrupo_receptor', 'total_iva_extragrupo_receptor'
        ]
        contribuyente_iva_filtered = format_money_columns(contribuyente_iva_filtered, money_columns_contribuyente_iva)

        contribuyente_iva_vertical = contribuyente_iva_filtered.melt(var_name="Atributo", value_name="Valor")

        # Gráfico de emisión para el contribuyente
        chart_iva_contribuyente = create_iva_comparison_chart(
            contribuyente_iva_vertical[contribuyente_iva_vertical['Atributo'].isin(['total_iva_intragrupo_emisor', 'total_iva_extragrupo_emisor'])],
            ['total_iva_intragrupo_emisor', 'total_iva_extragrupo_emisor'],
            "Comparación de IVA Intragrupo vs Extragrupo - Contribuyente"
        )

        # Gráfico de recepción
        chart_iva_receptor_louvain = create_iva_receptor_comparison_chart(
            iva_filtered_vertical[iva_filtered_vertical['Atributo'].isin(['recepcion_intragrupo', 'recepcion_extragrupo'])],
            ['recepcion_intragrupo', 'recepcion_extragrupo'],
            "Comparación de recepción IVA Intragrupo vs Extragrupo - Grupo Louvain"
        )

        chart_iva_receptor_contribuyente = create_iva_receptor_comparison_chart(
            contribuyente_iva_vertical[contribuyente_iva_vertical['Atributo'].isin(['total_iva_intragrupo_receptor', 'total_iva_extragrupo_receptor'])],
            ['total_iva_intragrupo_receptor', 'total_iva_extragrupo_receptor'],
            "Comparación de IVA Intragrupo vs Extragrupo - Receptor Contribuyente"
        )
        
        return (filtered_df_vertical, contribuyente_iva_vertical, iva_filtered_vertical,
                chart_iva_contribuyente, chart_iva_louvain,
                chart_iva_receptor_contribuyente, chart_iva_receptor_louvain)
    else:
        return pd.DataFrame(columns=contribuyentes.columns), pd.DataFrame(columns=iva.columns), pd.DataFrame(columns=contribuyente_iva.columns), None, None, None, None

# Crear la interfaz Gradio con Blocks
with gr.Blocks() as demo:
    
    with gr.Row():
        gr.Markdown("# Análisis de Contribuyentes - Servicio de Impuestos Internos de Chile", elem_id="header-title")
        gr.Image("logo-sii.png", label="", elem_id="logo-image", height=50, width=50)

    gr.Markdown("## Visualización de KPI para personas naturales y jurídicas")
    
    gr.Markdown("### Datos del Contribuyente")
    textbox_rut = gr.Textbox(placeholder="Introduce el RUT aquí...", label="RUT del Contribuyente")
    dataframe_contribuyentes = gr.Dataframe(label="Datos del Contribuyente")
    
    gr.Markdown("### Contribución del Contribuyente al Flujo Total del IVA del Grupo")
    dataframe_iva_contribuyente = gr.Dataframe(label="Contribución al IVA del Contribuyente al Grupo Louvain")
    
    # Gráficos de emisión y recepción para el contribuyente
    with gr.Row():
        plot_iva_contribuyente = gr.Plot(label="Comparación IVA Intragrupo vs Extragrupo - Contribuyente")
        plot_iva_receptor_contribuyente = gr.Plot(label="Comparación IVA Intragrupo vs Extragrupo - Receptor - Contribuyente")

    gr.Markdown("### Flujo Total del IVA para la Comunidad Louvain")
    dataframe_iva_louvain = gr.Dataframe(label="Flujo de IVA para la Comunidad Louvain")
    
    # Gráficos de emisión y recepción para el grupo Louvain
    with gr.Row():
        plot_iva_louvain = gr.Plot(label="Comparación IVA Intragrupo vs Extragrupo - Grupo Louvain")
        plot_iva_receptor_louvain = gr.Plot(label="Comparación IVA Intragrupo vs Extragrupo - Receptor - Grupo Louvain")
    # Cuarta Parte: Anexo con la información adicional de las tablas
    gr.Markdown("### Anexo: Información Adicional")
    
    # Mostrar la tabla contribuyente_iva
    gr.Markdown("#### Tabla: contribuyente_iva")
    gr.Markdown("""
    - contribuyente: Identificación del contribuyente.
    - total_iva_intragrupo_emisor: Total de IVA emitido por contribuyentes dentro del mismo grupo.
    - total_iva_extragrupo_emisor: Total de IVA emitido por contribuyentes fuera del grupo.
    - total_iva_intragrupo_receptor: Total de IVA recibido por contribuyentes dentro del mismo grupo.
    - total_iva_extragrupo_receptor: Total de IVA recibido por contribuyentes fuera del grupo.
    - arcos_intragrupo_emisor: Cantidad de conexiones comerciales desde las cuales se emitió IVA intragrupo.
    - arcos_extragrupo_emisor: Cantidad de conexiones comerciales desde las cuales se emitió IVA extragrupo.
    - arcos_intragrupo_receptor: Cantidad de conexiones comerciales desde las cuales se recibió IVA intragrupo.
    - arcos_extragrupo_receptor: Cantidad de conexiones comerciales desde las cuales se recibió IVA extragrupo.
    """)

    # Mostrar la tabla iva
    gr.Markdown("#### Tabla: iva")
    gr.Markdown("""
    - com: Identificador o código de la comunidad.
    - perct_emision_extra: Porcentaje de emisión fuera del grupo.
    - perct_recepcion_extra: Porcentaje de recepción fuera del grupo.
    - emision_extragrupo: Total de emisión fuera del grupo.
    - recepcion_extragrupo: Total de recepción fuera del grupo.
    - emision_intragrupo: Total de emisión dentro del grupo.
    - tasa_emision_extra_intra: Tasa de emisión extra dentro del grupo.
    - arcos_emision_intra: Cantidad de conexiones comerciales con emisión dentro del grupo.
    - arcos_emision_extra: Cantidad de conexiones comerciales con emisión fuera del grupo.
    - arcos_recepcion_intra: Cantidad de conexiones comerciales con recepción dentro del grupo.
    - arcos_recepcion_extra: Cantidad de conexiones comerciales con recepción fuera del grupo.
    - grupo_louvain: Grupo asignado según el algoritmo de Louvain basado en relaciones familiares, societarias y contables.
    - tamanio_comunidad: Tamaño del grupo en el que se encuentra el contribuyente.
    - ultimo_periodo_remanente_contaminado: Ultimo periodo de calculo de remanente contaminado.
    - remanente_cont: Monto de remanente contaminado.
    """)

    # Mostrar la tabla contribuyentes
    gr.Markdown("#### Tabla: contribuyentes")
    gr.Markdown("""
    - RUT_UNICO: Identificación única del contribuyente.
    - SOCIEDAD: Indica si el contribuyente es una sociedad (1) o no (0).
    - CONTADOR: Indica si el contribuyente es contador (1) o no (0).
    - NATURAL: Indica si el contribuyente es una persona natural (1) o no (0).
    - REPRESENTANTE: Indica si es un representante legal (1) o no (0).
    - Contaminacion: Relación del contribuyente con conexiones contaminadas, donde la contaminación proviene de personas que tienen alerta.
    - Total_pago_f29: Total de pago correspondiente a la declaración F29.
    - IVA_neto: Total de IVA neto a pagar.
    - Unidad_regional: Unidad geográfica del contribuyente.
    - Numero_documentos: Número total de documentos emitidos.
    - Tiempo_de_vida_dias: Días de actividad del contribuyente.
    - Alerta_inicial: Indicador de alerta inicial asociado al contribuyente.
    - Oscuridad: Medida de la "oscuridad" de la información disponible sobre el contribuyente.
    - Familiares: Número de familiares en la sociedad.
    - Personas_naturales_socios: Total de personas naturales que son socios.
    - Tasa_familiaridad: Grado de familiaridad entre los socios, donde 100% indica conexión familiar en el grupo.
    - Grupo_economico_inicial: Grupo económico inicial al que pertenece el contribuyente.
    - Grupo_louvain: Grupo asignado según el algoritmo de Louvain basado en relaciones.
    - Trabajadores_honorarios: Número de trabajadores contratados bajo honorarios.
    - Trabajadores_dependientes: Número de trabajadores dependientes.
    - Patrimonio_2022: Patrimonio declarado en el año 2022.
    """)
    textbox_rut.change(filter_data, inputs=textbox_rut, outputs=[
        dataframe_contribuyentes, 
        dataframe_iva_contribuyente, 
        dataframe_iva_louvain, 
        plot_iva_contribuyente, 
        plot_iva_louvain,
        plot_iva_receptor_contribuyente,
        plot_iva_receptor_louvain
    ])

# Lanzar la aplicación
demo.launch(share=True)



Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://55f62711be0b196426.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
